In [7]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00   1.30 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00   8.48 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  35.40 MB/s


In [9]:
import geocoder

In [13]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [14]:
latitude

43.6579524